# data cleansing notebook

## Objective:
Create a copy of the data that only meets MVP requirements

features (X):  latitude, longitude, victim age, victim descent (one-hot-encoded), victim_sex

target (y):  crime name

## 1. basic cleaning

## 1.1 Cleaning the data

In [1]:
import pandas as pd
import os
import numpy as np

from sklearn.preprocessing import OneHotEncoder


In [2]:
# fetch all the data from the raw_data folder
current_dir = os.getcwd()
current_dir

file_path = os.path.join(current_dir, '..', 'data', 'data.csv')
data = pd.read_csv(file_path)

In [3]:
data.head()

,division_number,date_reported,date_occurred,area,area_name,reporting_district,part,crime_code,crime_description,modus_operandi,...,status,status_description,crime_code_1,crime_code_2,crime_code_3,crime_code_4,location,cross_street,latitude,longitude
0,10304468,2020-01-08,2020-01-08 22:30:00,3,Southwest,377,2,624,BATTERY - SIMPLE ASSAULT,0444 0913,...,AO,Adult Other,624.0,NaN,NaN,NaN,1100 W 39TH PL,NaN,34.0141,-118.2978
1,190101086,2020-01-02,2020-01-01 03:30:00,1,Central,163,2,624,BATTERY - SIMPLE ASSAULT,0416 1822 1414,...,IC,Invest Cont,624.0,NaN,NaN,NaN,700 S HILL ST,NaN,34.0459,-118.2545
2,200110444,2020-04-14,2020-02-13 12:00:00,1,Central,155,2,845,SEX OFFENDER REGISTRANT OUT OF COMPLIANCE,1501,...,AA,Adult Arrest,845.0,NaN,NaN,NaN,200 E 6TH ST,NaN,34.0448,-118.2474
3,191501505,2020-01-01,2020-01-01 17:30:00,15,N Hollywood,1543,2,745,VANDALISM - MISDEAMEANOR ($399 OR UNDER),0329 1402,...,IC,Invest Cont,745.0,998.0,NaN,NaN,5400 CORTEEN PL,NaN,34.1685,-118.4019
4,191921269,2020-01-01,2020-01-01 04:15:00,19,Mission,1998,2,740,"VANDALISM - FELONY ($400 & OVER, ALL CHURCH VA...",0329,...,IC,Invest Cont,740.0,NaN,NaN,NaN,14400 TITUS ST,NaN,34.2198,-118.4468


In [43]:
#Removing illogical values and replacing erronous values

#removing negative age values
data=data[data['victim_age']>=0]

#removing null gender values
data = data[data['victim_sex'].notnull()]
data = data[data['victim_descent'].notnull()]


In [44]:
#exploring unique values for victim descent and victim sex columns
data['victim_sex'].unique(), data['victim_descent'].unique()

(array(['F', 'M', 'X', 'H', '-'], dtype=object),
 array(['B', 'H', 'X', 'W', 'A', 'O', 'C', 'F', 'K', 'I', 'V', 'Z', 'J',
        'P', 'G', 'U', 'D', 'S', 'L', '-'], dtype=object))

In [46]:
# from the above cell , '-' is present in both columns so we need to remove rows that contain it (they're not that many)
data=data[data['victim_sex'] != '-']
data=data[data['victim_descent'] != '-']


In [47]:
#checking the above code worked
data['victim_sex'].unique(), data['victim_descent'].unique()

(array(['F', 'M', 'X', 'H'], dtype=object),
 array(['B', 'H', 'X', 'W', 'A', 'O', 'C', 'F', 'K', 'I', 'V', 'Z', 'J',
        'P', 'G', 'U', 'D', 'S', 'L'], dtype=object))

In [48]:
# Parse datetime string to datetime object
data['date_occurred'] = pd.to_datetime(data['date_occurred'])

# Create new columns for date and time
data['date_column'] = data['date_occurred'].dt.date
data['time_column'] = data['date_occurred'].dt.time

In [49]:
data.head()

,division_number,date_reported,date_occurred,area,area_name,reporting_district,part,crime_code,crime_description,modus_operandi,victim_age,victim_sex,victim_descent,premise_code,premise_description,weapon_code,weapon_description,status,status_description,crime_code_1,crime_code_2,crime_code_3,crime_code_4,location,cross_street,latitude,longitude,date_column,time_column,time_category
0,10304468,2020-01-08,2020-01-08 22:30:00,3,Southwest,377,2,624,BATTERY - SIMPLE ASSAULT,0444 0913,36,F,B,501.0,SINGLE FAMILY DWELLING,400.0,"STRONG-ARM (HANDS, FIST, FEET OR BODILY FORCE)",AO,Adult Other,624.0,NaN,NaN,NaN,1100 W 39TH PL,NaN,34.0141,-118.2978,2020-01-08,22:30:00,evening
1,190101086,2020-01-02,2020-01-01 03:30:00,1,Central,163,2,624,BATTERY - SIMPLE ASSAULT,0416 1822 1414,25,M,H,102.0,SIDEWALK,500.0,UNKNOWN WEAPON/OTHER WEAPON,IC,Invest Cont,624.0,NaN,NaN,NaN,700 S HILL ST,NaN,34.0459,-118.2545,2020-01-01,03:30:00,night
2,200110444,2020-04-14,2020-02-13 12:00:00,1,Central,155,2,845,SEX OFFENDER REGISTRANT OUT OF COMPLIANCE,1501,0,X,X,726.0,POLICE FACILITY,NaN,NaN,AA,Adult Arrest,845.0,NaN,NaN,NaN,200 E 6TH ST,NaN,34.0448,-118.2474,2020-02-13,12:00:00,afternoon
3,191501505,2020-01-01,2020-01-01 17:30:00,15,N Hollywood,1543,2,745,VANDALISM - MISDEAMEANOR ($399 OR UNDER),0329 1402,76,F,W,502.0,"MULTI-UNIT DWELLING (APARTMENT, DUPLEX, ETC)",NaN,NaN,IC,Invest Cont,745.0,998.0,NaN,NaN,5400 CORTEEN PL,NaN,34.1685,-118.4019,2020-01-01,17:30:00,afternoon
4,191921269,2020-01-01,2020-01-01 04:15:00,19,Mission,1998,2,740,"VANDALISM - FELONY ($400 & OVER, ALL CHURCH VA...",0329,31,X,X,409.0,BEAUTY SUPPLY STORE,NaN,NaN,IC,Invest Cont,740.0,NaN,NaN,NaN,14400 TITUS ST,NaN,34.2198,-118.4468,2020-01-01,04:15:00,night


In [50]:
# Create new column 'time_category' based on time
def categorize_time(time):
    hour = time.hour
    if 6 <= hour < 12:
        return 'morning'
    elif 12 <= hour < 18:
        return 'afternoon'
    elif 18 <= hour < 24:
        return 'evening'
    else:
        return 'night'

data['time_category'] = data['time_column'].apply(categorize_time)

## 1.2 Selecting the features and the target

In [59]:
columns_to_keep=['victim_age','victim_sex','victim_descent','latitude','longitude','date_column','time_column','time_category']
X=data.loc[:,columns_to_keep]


In [52]:
X.head()

,victim_age,victim_sex,victim_descent,latitude,longitude,date_column,time_column,time_category
0,36,F,B,34.0141,-118.2978,2020-01-08,22:30:00,evening
1,25,M,H,34.0459,-118.2545,2020-01-01,03:30:00,night
2,0,X,X,34.0448,-118.2474,2020-02-13,12:00:00,afternoon
3,76,F,W,34.1685,-118.4019,2020-01-01,17:30:00,afternoon
4,31,X,X,34.2198,-118.4468,2020-01-01,04:15:00,night


In [53]:
y=data['crime_code']

## 1.3 One-hot-encoding victim_descent and victim_sex columns

In [54]:
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)

In [55]:
X_encoded = pd.get_dummies(X, columns=['victim_descent', 'victim_sex'])

In [56]:
X_encoded.head()

,victim_age,latitude,longitude,date_column,time_column,time_category,victim_descent_A,victim_descent_B,victim_descent_C,victim_descent_D,victim_descent_F,victim_descent_G,victim_descent_H,victim_descent_I,victim_descent_J,victim_descent_K,victim_descent_L,victim_descent_O,victim_descent_P,victim_descent_S,victim_descent_U,victim_descent_V,victim_descent_W,victim_descent_X,victim_descent_Z,victim_sex_F,victim_sex_H,victim_sex_M,victim_sex_X
0,36,34.0141,-118.2978,2020-01-08,22:30:00,evening,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False
1,25,34.0459,-118.2545,2020-01-01,03:30:00,night,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False
2,0,34.0448,-118.2474,2020-02-13,12:00:00,afternoon,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,True
3,76,34.1685,-118.4019,2020-01-01,17:30:00,afternoon,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,True,False,False,False
4,31,34.2198,-118.4468,2020-01-01,04:15:00,night,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,True


In [57]:
X_encoded = X_encoded.replace({True: 1, False: 0})

/tmp/ipykernel_597912/1959818255.py:1: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  X_encoded = X_encoded.replace({True: 1, False: 0})


In [58]:
X_encoded.head()

,victim_age,latitude,longitude,date_column,time_column,time_category,victim_descent_A,victim_descent_B,victim_descent_C,victim_descent_D,victim_descent_F,victim_descent_G,victim_descent_H,victim_descent_I,victim_descent_J,victim_descent_K,victim_descent_L,victim_descent_O,victim_descent_P,victim_descent_S,victim_descent_U,victim_descent_V,victim_descent_W,victim_descent_X,victim_descent_Z,victim_sex_F,victim_sex_H,victim_sex_M,victim_sex_X
0,36,34.0141,-118.2978,2020-01-08,22:30:00,evening,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0
1,25,34.0459,-118.2545,2020-01-01,03:30:00,night,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0
2,0,34.0448,-118.2474,2020-02-13,12:00:00,afternoon,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1
3,76,34.1685,-118.4019,2020-01-01,17:30:00,afternoon,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0
4,31,34.2198,-118.4468,2020-01-01,04:15:00,night,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1
